In [16]:
import pandas as pd
import yaml
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# ---------- Paths ----------
geo_path = '../data/processed/ageb_merged/national_ageb_geo_full.parquet'
weights_path = '../config/weights.yaml'
output_path = '../data/processed/indices/sci_national.parquet'

# ---------- Load ----------
ageb_geo = gpd.read_parquet(geo_path)
print(f"Loaded {len(ageb_geo)} geo features.")

# Filter out empty AGEBs (avoid meaningless SCI)
ageb_geo['POBTOT'] = pd.to_numeric(ageb_geo.get('POBTOT', np.nan), errors='coerce')
ageb_geo = ageb_geo[ageb_geo['POBTOT'] > 0].copy()
print(f"After filtering empty AGEBs: {len(ageb_geo)} features.")

# ---------- Coerce numeric types ----------
numeric_cols = [
    # Education
    'GRAPROES', 'P15YM_AN', 'P18YM_PB', 'P_15YMAS', 'P_18YMAS',
    # Employment
    'POCUPADA', 'PEA', 'P_12YMAS',
    # Goods/assets
    'VPH_REFRI', 'VPH_PC', 'VPH_AUTOM', 'VPH_LAVAD',
    # Housing & services
    'VPH_PISODT', 'VPH_AGUADV', 'VPH_C_ELEC', 'VPH_DRENAJ', 'VPH_TELEF',
    'TVIVHAB',
    # Overcrowding
    'PRO_OCUP_C',
    # Dependency
    'POB0_14', 'POB65_MAS',
    # Household structure
    'HOGJEF_F', 'TOTHOG',
    # Optional
    'REL_H_M',
    # Grouping
    'ENTIDAD'
]
for col in numeric_cols:
    if col in ageb_geo.columns:
        ageb_geo[col] = pd.to_numeric(ageb_geo[col], errors='coerce')

print("Numeric coercion complete.")

# ---------- Load weights ----------
with open(weights_path, 'r', encoding='utf-8') as f:
    weights = yaml.safe_load(f)

print("Weights loaded:", weights)

# =========================
# Helpers
# =========================
def safe_pct(num, denom):
    "Return percent (0-100) safely."
    if pd.isna(num) or pd.isna(denom) or denom == 0:
        return np.nan
    return (num / denom) * 100.0

def winsorize_series(s, lower_q=0.02, upper_q=0.98):
    "Clip series to [q_low, q_high] to reduce outlier influence."
    s = pd.to_numeric(s, errors='coerce').astype(float)
    q_low = np.nanquantile(s, lower_q)
    q_high = np.nanquantile(s, upper_q)
    return s.clip(q_low, q_high)

def robust_minmax(s, lower_q=0.02, upper_q=0.98):
    "Winsorize then min-max scale to [0,1]."
    s2 = winsorize_series(s, lower_q, upper_q)
    smin = np.nanmin(s2)
    smax = np.nanmax(s2)
    if not np.isfinite(smin) or not np.isfinite(smax) or smax == smin:
        return pd.Series(np.nan, index=s.index)
    return (s2 - smin) / (smax - smin)

def mean_ignore_nan(*arrs):
    stacked = np.vstack([np.array(a, dtype=float) for a in arrs])
    return np.nanmean(stacked, axis=0)

# =========================
# 1) RAW indicators (interpretable)
# =========================

# ---- Education ----
ageb_geo['edu_graproes_raw'] = ageb_geo.get('GRAPROES')  # mean years of schooling

ageb_geo['edu_illiteracy_pct'] = ageb_geo.apply(
    lambda r: safe_pct(r.get('P15YM_AN'), r.get('P_15YMAS')),
    axis=1
)

ageb_geo['edu_postbasic_pct'] = ageb_geo.apply(
    lambda r: safe_pct(r.get('P18YM_PB'), r.get('P_18YMAS')),
    axis=1
)

# ---- Employment ----
def employment_pct(row):
    num = row.get('POCUPADA')
    denom = row.get('PEA')
    if pd.notna(denom) and denom != 0:
        return safe_pct(num, denom)
    return safe_pct(num, row.get('P_12YMAS'))

ageb_geo['employment_pct'] = ageb_geo.apply(employment_pct, axis=1)

# ---- Goods/assets ----
goods_vars = ['VPH_REFRI', 'VPH_PC', 'VPH_AUTOM', 'VPH_LAVAD']
for v in goods_vars:
    ageb_geo[f'{v}_pct'] = ageb_geo.apply(lambda r, vv=v: safe_pct(r.get(vv), r.get('TVIVHAB')), axis=1)
ageb_geo['poverty_goods_pct'] = ageb_geo[[f'{v}_pct' for v in goods_vars]].mean(axis=1, skipna=True)

# ---- Services access ----
svc_vars = ['VPH_AGUADV', 'VPH_C_ELEC', 'VPH_DRENAJ', 'VPH_TELEF']
for v in svc_vars:
    ageb_geo[f'{v}_pct'] = ageb_geo.apply(lambda r, vv=v: safe_pct(r.get(vv), r.get('TVIVHAB')), axis=1)
ageb_geo['services_access_pct'] = ageb_geo[[f'{v}_pct' for v in svc_vars]].mean(axis=1, skipna=True)

# ---- Housing: dirt floor (badness) ----
ageb_geo['dirt_floor_pct'] = ageb_geo.apply(lambda r: safe_pct(r.get('VPH_PISODT'), r.get('TVIVHAB')), axis=1)

# ---- Overcrowding: persons per room (badness) ----
ageb_geo['overcrowding_raw'] = ageb_geo.get('PRO_OCUP_C')

# ---- Dependency ratio (badness) ----
ageb_geo['dependency_pct'] = ageb_geo.apply(
    lambda r: safe_pct((r.get('POB0_14') or 0) + (r.get('POB65_MAS') or 0), r.get('POBTOT')),
    axis=1
)

# ---- Female-headed households (badness-ish) ----
ageb_geo['female_headed_pct'] = ageb_geo.apply(lambda r: safe_pct(r.get('HOGJEF_F'), r.get('TOTHOG')), axis=1)

# ---- Gender balance deviation (optional) ----
ageb_geo['gender_balance_dev'] = np.abs(ageb_geo.get('REL_H_M') - 100)

# ---- Marginalization (UPDATED): deprivation proxy (badness) ----
# Assumes VPH_* are counts of households WITH service.
ageb_geo['lack_water_pct'] = 100 - ageb_geo['VPH_AGUADV_pct']
ageb_geo['lack_elec_pct'] = 100 - ageb_geo['VPH_C_ELEC_pct']
ageb_geo['lack_drain_pct'] = 100 - ageb_geo['VPH_DRENAJ_pct']
ageb_geo['lack_phone_pct'] = 100 - ageb_geo['VPH_TELEF_pct']

ageb_geo['marginalization_deprivation_pct'] = ageb_geo[
    ['lack_water_pct', 'lack_elec_pct', 'lack_drain_pct', 'lack_phone_pct', 'dirt_floor_pct']
].mean(axis=1, skipna=True)

# =========================
# 2) Normalize to 0–1
# =========================

# Education: normalize components separately, then combine
ageb_geo['edu_graproes_norm'] = robust_minmax(ageb_geo['edu_graproes_raw'])          # goodness
ageb_geo['edu_postbasic_norm'] = robust_minmax(ageb_geo['edu_postbasic_pct'])        # goodness
ageb_geo['edu_illiteracy_bad'] = robust_minmax(ageb_geo['edu_illiteracy_pct'])       # badness

ageb_geo['education_norm'] = (
    mean_ignore_nan(ageb_geo['edu_graproes_norm'], ageb_geo['edu_postbasic_norm'])
    - 0.5 * ageb_geo['edu_illiteracy_bad']
).clip(0, 1)

# Positives (goodness)
ageb_geo['employment_norm'] = robust_minmax(ageb_geo['employment_pct'])
ageb_geo['poverty_goods_norm'] = robust_minmax(ageb_geo['poverty_goods_pct'])
ageb_geo['services_access_norm'] = robust_minmax(ageb_geo['services_access_pct'])

# Housing goodness = 1 - dirt-floor badness
ageb_geo['dirt_floor_bad'] = robust_minmax(ageb_geo['dirt_floor_pct'])
ageb_geo['housing_quality_norm'] = (1 - ageb_geo['dirt_floor_bad']).clip(0, 1)

# Penalties (badness; DO NOT invert)
ageb_geo['overcrowding_norm'] = robust_minmax(ageb_geo['overcrowding_raw'])
ageb_geo['dependency_norm'] = robust_minmax(ageb_geo['dependency_pct'])
ageb_geo['female_headed_norm'] = robust_minmax(ageb_geo['female_headed_pct'])
ageb_geo['marginalization_norm'] = robust_minmax(ageb_geo['marginalization_deprivation_pct'])

# Optional gender balance (goodness where 1 is best). Weight is usually 0.
ageb_geo['gender_balance_norm'] = (1 - robust_minmax(ageb_geo['gender_balance_dev'])).clip(0, 1)

# =========================
# 3) Compute SCI (weighted sum)
# =========================
norm_map = {
    'education': 'education_norm',
    'employment': 'employment_norm',
    'poverty_goods': 'poverty_goods_norm',
    'housing_quality': 'housing_quality_norm',
    'services_access': 'services_access_norm',
    'overcrowding': 'overcrowding_norm',
    'marginalization': 'marginalization_norm',
    'dependency': 'dependency_norm',
    'female_headed': 'female_headed_norm',
    'gender_balance': 'gender_balance_norm',
}

ageb_geo['SCI_raw'] = 0.0
for cat, w in weights.items():
    col = norm_map.get(cat)
    if col is None or col not in ageb_geo.columns:
        print(f"WARNING: weight category '{cat}' has no computed column; skipping.")
        continue
    ageb_geo['SCI_raw'] += ageb_geo[col] * float(w)

ageb_geo['SCI'] = (ageb_geo['SCI_raw'] * 100).clip(0, 100).round(2)

print("SCI summary:")
print(ageb_geo['SCI'].describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99]).round(2))

if 'ENTIDAD' in ageb_geo.columns and 'NOM_ENT' in ageb_geo.columns:
    state_stats = ageb_geo.groupby(['ENTIDAD', 'NOM_ENT'])['SCI'].agg(['min', 'max', 'mean']).round(1)
    state_stats.columns = ['Min', 'Max', 'Mean']

    print("\n=== SCI Statistics by State ===")
    for (ent, nom_ent), row in state_stats.iterrows():
        try:
            ent_fmt = int(ent)
        except Exception:
            ent_fmt = ent
        print(f"State {ent_fmt} ({nom_ent}): {row['Min']}–{row['Max']} (mean {row['Mean']})")

# =========================
# 5) Export
# =========================
ageb_geo.to_parquet(output_path, index=False)
print(f"Exported SCI to {output_path} (shape: {ageb_geo.shape}).")


Loaded 63616 geo features.
After filtering empty AGEBs: 61226 features.
Numeric coercion complete.
Weights loaded: {'education': 0.34, 'employment': 0.22, 'poverty_goods': 0.15, 'housing_quality': 0.12, 'services_access': 0.17, 'overcrowding': -0.1, 'marginalization': -0.05, 'dependency': -0.05, 'female_headed': -0.03, 'gender_balance': 0.0}


/var/folders/vj/97q__v7n1rb7jd35sbnf6bch0000gn/T/ipykernel_32584/2868694291.py:82: RuntimeWarning: Mean of empty slice
  return np.nanmean(stacked, axis=0)


SCI summary:
count    48070.00
mean        40.51
std         15.90
min          0.00
1%           6.46
5%          16.16
25%         29.50
50%         39.41
75%         50.53
95%         69.87
99%         78.94
max         90.66
Name: SCI, dtype: float64

=== SCI Statistics by State ===
State 1 (Aguascalientes): 6.0–82.2 (mean 46.6)
State 2 (Baja California): 0.0–85.2 (mean 47.4)
State 3 (Baja California Sur): 0.0–83.4 (mean 47.5)
State 4 (Campeche): 6.8–82.4 (mean 38.3)
State 5 (Coahuila de Zaragoza): 0.0–85.6 (mean 44.0)
State 6 (Colima): 15.1–80.4 (mean 44.3)
State 7 (Chiapas): 0.0–83.9 (mean 27.7)
State 8 (Chihuahua): 0.0–87.3 (mean 43.8)
State 9 (Ciudad de México): 15.6–83.1 (mean 54.8)
State 10 (Durango): 0.0–83.3 (mean 39.1)
State 11 (Guanajuato): 0.0–83.4 (mean 38.6)
State 12 (Guerrero): 0.0–76.8 (mean 31.1)
State 13 (Hidalgo): 0.0–79.8 (mean 40.7)
State 14 (Jalisco): 0.0–85.6 (mean 43.0)
State 15 (México): 0.0–86.1 (mean 43.5)
State 16 (Michoacán de Ocampo): 0.0–82.6 (mean 34.